In [3]:
import numpy as np
import matplotlib.pyplot as plt
from imaids.models import DeltaSabia
from idanalysis.analysis import Tools
import utils
import imaids
from mathphys.functions import save_pickle, load_pickle
from scipy.integrate import cumtrapz


In [4]:
def generate_radia_model(phase=0, dgv=0, trf_on_blocks=False):
    """."""

    delta = DeltaSabia(trf_on_blocks=trf_on_blocks)
    delta.set_cassete_positions(dp=phase, dgv=dgv)

    return delta


In [5]:
def get_cassettes_names(model):
    return list(model.cassettes_ref.keys())


In [6]:
def get_fieldmap(phase, dgv):
    fmap_fname = Tools.get_fmap_fname(utils.ID_CONFIGS, utils.MEAS_DATA_PATH,
                                      config_keys=(phase, dgv),
                                      config_dict=utils.CONFIG_DICT)
    return Tools.get_fmap(fmap_fname)


In [7]:
def get_full_field(fmap):
    full_field = np.zeros((len(fmap.rx), len(fmap.rz), 3))
    full_field[:, :, 0]  = fmap.bx[0]
    full_field[:, :, 1]  = fmap.by[0]
    full_field[:, :, 2]  = fmap.bz[0]
    return full_field


In [8]:
def get_onaxis_field(rx, rz, fmap):
    full_field = get_full_field(rx, rz, fmap)
    field_onaxis = full_field[fmap.rx_zero,:, :]
    return field_onaxis


In [9]:
def get_onaxis_field_model(rz, model):
    field_onaxis = np.zeros((len(rz), 3))
    field_onaxis[:, :] = model.get_field(x=0, y=0, z=rz, nproc=24)
    return field_onaxis


In [10]:
def get_full_field_model(rx, rz, model):
    field_model = np.zeros((len(rx), len(rz), 3))
    for i, x in enumerate(rx):
        field_model[i, :, :] = model.get_field(x=x, y=0, z=rz, nproc=24)
    return field_model


In [11]:
imaids.utils.set_len_tol(5e-10, 5e-10)
delta = generate_radia_model(phase=0, dgv=0)
cas_names = get_cassettes_names(delta)

fmap = get_fieldmap(phase=0, dgv=0)
rx = fmap.rx
ry = fmap.ry
rz = fmap.rz


ID4818


In [11]:
# field_onaxis_model = get_onaxis_field_model(rz, delta)
# field_onaxis_meas = get_onaxis_field(rx, rz, fmap)
# plt.plot(field_onaxis_meas[:, 2] - field_onaxis_model[:, 2])
# plt.show()


# Construct response matrix for all diferent blocks of a cassette

In [12]:
def do_block_displacement(delta, block):
    block.shift([0, -delta.gap/2, 0])
    center = np.array(block.center_point)
    center[0] = 0
    center[1] = 0
    block.shift(-center)
    block.rotate([0, 0, 0], [0, 0, 1], -np.pi/4)


In [13]:
def calculate_jac_elements(delta, rx, rz):
    blocks = [
        delta.cassettes_ref['cid'].blocks[block_nr] for block_nr in
        np.arange(0, 8, 1)]
    blocks += [
            delta.cassettes_ref['cid'].blocks[-4],
            delta.cassettes_ref['cid'].blocks[-2]]
    nr_blocks = len(blocks)

    x = rx
    z = rz

    dmag = 0.02
    delta_mags = [np.array([dmag, 0, 0]),
                  np.array([0, dmag, 0]),
                  np.array([0, 0, dmag]),
                  ]

    jac_elem = np.zeros((len(z), len(x), len(delta_mags), nr_blocks, 3))

    for k, block in enumerate(blocks):
        print('block: ', k)
        for i, x_ in enumerate(x):
            for j, d_mag in enumerate(delta_mags):
                block.magnetization = np.array(block.magnetization) + d_mag
                do_block_displacement(delta, block)
                field_p = block.get_field(x=x_, y=0, z=z, nproc=24)

                block.magnetization = np.array(block.magnetization) - 2*d_mag
                do_block_displacement(delta, block)
                field_n = block.get_field(x=x_, y=0, z=z, nproc=24)

                block.magnetization = np.array(block.magnetization) + d_mag
                do_block_displacement(delta, block)
                diff = (field_p - field_n)/(2*dmag)
                half_size = int(len(z)/2)
                diff[0:half_size-600] = 0
                diff[half_size+600:] = 0

                jac_elem[:, i, j, k, :] = diff
    return jac_elem


In [14]:
# def calculate_jac_elements(delta):
#     blocks = [delta.cassettes_ref['cid'].blocks[block_nr] for block_nr in np.arange(0,6,2)]
#     nr_blocks = len(blocks)

#     x = rx
#     z = rz

#     dmag = 0.02
#     delta_mags = [np.array([dmag, 0, 0]),
#                   np.array([0, dmag, 0]),
#                   np.array([0, 0, dmag]),
#                   ]

#     jac_elem = np.zeros((len(z), len(x), len(delta_mags), nr_blocks, 3))

#     for k, block in enumerate(blocks):
#         print('block: ', k)
#         for i, x_ in enumerate(x):
#             for j, delta_mag in enumerate(delta_mags):
#                 block.magnetization = np.array(block.magnetization) + delta_mag
#                 do_block_displacement(delta, block)
#                 field_p = block.get_field(x=x_, y=0, z=z, nproc=24)

#                 block.magnetization = np.array(block.magnetization) - 2*delta_mag
#                 do_block_displacement(delta, block)
#                 field_n = block.get_field(x=x_, y=0, z=z, nproc=24)

#                 block.magnetization = np.array(block.magnetization) + delta_mag
#                 do_block_displacement(delta, block)
#                 diff = (field_p - field_n)/(2*dmag)
#                 half_size = int(len(z)/2)
#                 diff[0:half_size-600] = 0
#                 diff[half_size+600:] = 0

#                 jac_elem[:, i, j, k, :] = diff
#     return jac_elem


In [15]:
def transform_ci_2_cs(jac_elem):
    # delta x
    jac_elem[:, :, 0, :, 0] = -1*jac_elem[:, :, 0, :, 0]  # Change Bx signal
    jac_elem[:, :, 0, :, 2] = -1*jac_elem[:, :, 0, :, 2]  # Change Bz signal

    # delta y
    jac_elem[:, :, 1, :, 1] = -1*jac_elem[:, :, 1, :, 1]  # Change By signal

    # delta z
    jac_elem[:, :, 2, :, 1] = -1*jac_elem[:, :, 2, :, 1]  # Change By signal
    return jac_elem

def transform_cd_2_ce(jac_elem):
    jac_elem[:, :, :, :, :] = jac_elem[:, ::-1, :, :, :]  # Invert x grid

    # delta x
    jac_elem[:, :, 0, :, 1] = -1*jac_elem[:, :, 0, :, 1]  # Change By signal
    jac_elem[:, :, 0, :, 2] = -1*jac_elem[:, :, 0, :, 2]  # Change Bz signal

    # delta y
    jac_elem[:, :, 1, :, 0] = -1*jac_elem[:, :, 1, :, 0]  # Change Bx signal

    # delta z
    jac_elem[:, :, 2, :, 0] = -1*jac_elem[:, :, 2, :, 0]  # Change Bx signal
    return jac_elem

def transform_cid_2_cse(jac_elem):
    jac_cie = transform_cd_2_ce(jac_elem)  # Transform to cie
    jac_cse = transform_ci_2_cs(jac_cie)  # Transform to cse
    return jac_cse


In [16]:
def create_pre_jacobian(jac_elem):
    cid = jac_elem.copy()
    cie = jac_elem.copy()
    csd = jac_elem.copy()
    cse = jac_elem.copy()

    csd = transform_ci_2_cs(csd)
    cie = transform_cd_2_ce(cie)
    cse = transform_cid_2_cse(cse)

    pre_jacobians = {
                     'cse': cse,
                     'csd': csd,
                     'cie': cie,
                     'cid': cid,
                    }
    return pre_jacobians


In [17]:
def get_block_type(block_number, nr_blocks):
    if block_number <= 3:
        block_type = block_number
    elif block_number <= nr_blocks-5:
        block_type = (block_number-4) % 4 + 4
    else:
        cont = -1*(nr_blocks-block_number-4)
        if cont % 2 != 0:
            block_type = nr_blocks-block_number-1
        elif cont == 0:
            block_type = 8
        else:
            block_type = 9
    return block_type


In [ ]:
# def get_block_type(block_number, nr_blocks):
#     block_selector = 2*np.ones(nr_blocks)
#     block_selector[0:2] = 0
#     block_selector[nr_blocks-2:] = 0
#     block_selector[2:4] = 1
#     block_selector[nr_blocks-4:nr_blocks-2] = 1
#     block_type = block_selector[block_number]
#     return int(block_type)


In [ ]:
for j, block in enumerate(delta.cassettes_ref['cid'].blocks):
    block_type = get_block_type(j, len(delta.cassettes_ref['cid'].blocks))
    print(block.length, block.magnetization, block_type)


In [ ]:
def create_jacobian(pre_jacobians, delta):
    nr_blocks = delta.cassettes_ref['cid'].nr_blocks
    z_step = np.diff(rz)[0]
    jacobian = np.zeros((len(rz)*len(rx)*3, 4*nr_blocks*3))
    cas_names = get_cassettes_names(delta)
    for i, cas_name in enumerate(cas_names):
        cas_shift = delta.cassettes_ref[cas_name].center_point[-1]
        for j, block in enumerate(delta.cassettes_ref[cas_name].blocks):
            block_shift = block.center_point[-1]
            bl_type = get_block_type(j, nr_blocks)
            idx = np.argmin(np.abs(rz-(block_shift + cas_shift)))
            shift = idx - np.argmin(np.abs(rz))
            shifted_prejac = np.roll(pre_jacobians[cas_name], shift, 0)
            for mag in np.arange(3):
                column = 3*i*nr_blocks + 3*j + mag
                jacobian[:, column] = shifted_prejac[:, :, mag, bl_type, :].ravel(order='F')
    return jacobian


#### order of jacobian columns: z -> x -> b

#### order of jacobian lines: mag -> block -> cassette

In [ ]:
def calc_inverse_jacobian(jacobian):
    u, s, vt = np.linalg.svd(jacobian, full_matrices=False)
    tol_svals = 1e-5
    sel_svals = abs(s) > tol_svals
    ismat = np.zeros(s.shape)
    ismat[sel_svals] = 1/s[sel_svals]
    ismat = np.diag(ismat)
    invmat = np.dot(np.dot(vt.T, ismat), u.T)
    return invmat, u, s, vt


In [ ]:
def calc_field_difference(full_field_meas):
    full_field_model = get_full_field_model(rx, rz, delta)
    full_field_model = np.swapaxes(full_field_model, 0, 1)
    field_diff = (full_field_meas - full_field_model).ravel(order='F')
    return field_diff


In [ ]:
# Code for tests with other model
phase, dgv = 0, 0
delta2 = generate_radia_model(phase=phase, dgv=dgv)
mag_dict = dict()
for i, cas_name in enumerate(cas_names):
        mag_list = list()
        for j, block in enumerate(delta2.cassettes_ref[cas_name].blocks):
                mag_list.append(1e0*np.array(block.magnetization) + 1*np.random.rand(3))
        mag_dict[cas_name] = mag_list
delta2.create_radia_object(magnetization_dict=mag_dict)
delta2.set_cassete_positions(dp=phase, dgv=dgv)
full_field_meas = get_full_field_model(rx, rz, delta2)
full_field_meas = np.swapaxes(full_field_meas, 0, 1)


In [ ]:
phase, dgv = 0, 0
imaids.utils.set_len_tol(5e-10, 5e-10)
delta = generate_radia_model(phase=phase, dgv=dgv)
jac_elem = load_pickle('jac_elements_v0.pickle')
pre_jacobians = create_pre_jacobian(jac_elem)
jacobian = create_jacobian(pre_jacobians, delta)
invmat, u, s, vt = calc_inverse_jacobian(jacobian)


In [ ]:
#  create radia model
phase, dgv = -3.14, 15
imaids.utils.set_len_tol(5e-10, 5e-10)
delta_nominal = generate_radia_model(phase=phase, dgv=dgv)
cas_names = get_cassettes_names(delta)

full_field_model0 = get_full_field_model(rx, rz, delta_nominal)
full_field_model0 = np.swapaxes(full_field_model0, 0, 1)


In [ ]:
#  create radia model
phase, dgv = 0, 0
imaids.utils.set_len_tol(5e-10, 5e-10)
delta = generate_radia_model(phase=phase, dgv=dgv)
cas_names = get_cassettes_names(delta)

# Load fmap
# fmap = get_fieldmap(phase=phase, dgv=dgv)
# full_field_meas = get_full_field(fmap)
# full_field_meas = np.swapaxes(full_field_meas, 0, 1)
# rx = fmap.rx
# ry = fmap.ry
# rz = fmap.rz
nr_blocks = delta.cassettes_ref[cas_names[0]].nr_blocks

# Calc jacobian
# jac_elem = calculate_jac_elements(delta)
# save_pickle(jac_elem, 'jac_elements_3blocks', overwrite=True)
jac_elem = load_pickle('jac_elements_v0.pickle')
pre_jacobians = create_pre_jacobian(jac_elem)
jacobian = create_jacobian(pre_jacobians, delta)
invmat, u, s, vt = calc_inverse_jacobian(jacobian)

residue = list()
for k in range(15):
    print('iteraction: ', k)
    field_diff = calc_field_difference(full_field_meas)
    residue_ = np.std(field_diff)
    residue.append(residue_)
    print(residue_)
    # idx = np.argwhere(np.abs(field_diff) < 1e-6)
    # field_diff[idx] = 0

    deltas = np.dot(invmat, field_diff)
    deltas = deltas.reshape((len(cas_names), nr_blocks, 3))
    mag_dict = dict()
    cas_names = get_cassettes_names(delta)
    for i, cas_name in enumerate(cas_names):
        mag_list = list()
        for j, block in enumerate(delta.cassettes_ref[cas_name].blocks):
            mag_list.append( np.array(block.magnetization) + deltas[i, j, :])
        mag_dict[cas_name] = mag_list
    delta.create_radia_object(magnetization_dict=mag_dict)
    delta.set_cassete_positions(dp=phase, dgv=dgv)


In [ ]:
full_field_model = get_full_field_model(rx, rz, delta)
full_field_model = np.swapaxes(full_field_model, 0, 1)
field_diff = (full_field_meas - full_field_model).ravel(order='F')
residue_ = np.std(field_diff)
print(residue_)
residue.append(residue_)


In [ ]:
%matplotlib qt5
comp = 1
plt.plot(rz, full_field_model0[:, 0, comp], label='model nominal')
plt.plot(rz, full_field_model[:, 0, comp], label='model fitted')
plt.plot(rz, full_field_meas[:, 0, comp], label='meas')
plt.xlabel('z [mm]')
plt.ylabel('B [T]')
plt.legend()


In [ ]:
%matplotlib qt5
comp = 0
plane = 10
diff = -full_field_meas[:, plane, comp] + full_field_model[:, plane, comp]


kick_res = 1e5*np.trapz(rz*1e-3, diff)
print(kick_res)

plt.plot(rz, full_field_meas[:, plane, comp], label='meas')
plt.plot(rz, full_field_model[:, plane, comp], label='model_1')
# plt.plot(rz, diff, label='Diff')
plt.xlabel('z [mm]')
plt.ylabel('B [T]')
plt.legend()


In [ ]:
diff = -full_field_meas[:, plane, comp] + full_field_model[:, plane, comp]
kick_res = 1e5*np.trapz(rz*1e-3, diff)
print(kick_res)

plt.plot(rz, diff, label='Diff')
plt.xlabel('z [mm]')
plt.ylabel('B [T]')
plt.legend()


In [ ]:
phase, dgv = 0, 0
p = str(int(np.modf(np.abs(phase))[-1]))
gv = str(int(np.modf(np.abs(dgv))[-1]))
mag_fname = './mags/p{}_dgv{}.pickle'.format(p, gv)
print(mag_fname)


In [ ]:
delta = DeltaSabia()
delta.set_cassete_positions(dp=phase, dgv=dgv)
full_field_model0 = get_full_field_model(rx, rz, delta)
full_field_model0 = np.swapaxes(full_field_model0, 0, 1)


In [ ]:
mag_dict = load_pickle(mag_fname)
delta = DeltaSabia()
delta.create_radia_object(magnetization_dict=mag_dict)
delta.set_cassete_positions(dp=phase, dgv=dgv)
full_field_model = get_full_field_model(rx, rz, delta)
full_field_model = np.swapaxes(full_field_model, 0, 1)


In [ ]:
fmap = get_fieldmap(phase=phase, dgv=dgv)
full_field_meas = get_full_field(fmap)
full_field_meas = np.swapaxes(full_field_meas, 0, 1)


In [ ]:
%matplotlib qt5
fig, axs = plt.subplots(3, 1, sharex=True)
fig.set_size_inches(15, 8)

xpos = 0

diffx = full_field_meas[:, xpos, 0] - full_field_model[:, xpos, 0]
diffy = full_field_meas[:, xpos, 1] - full_field_model[:, xpos, 1]
diffz = full_field_meas[:, xpos, 2] - full_field_model[:, xpos, 2]

ix = cumtrapz(diffx)
iy = cumtrapz(diffy)
iz = cumtrapz(diffz)

show_diff = True

axs[0].set_title('On-axis fields')
if not show_diff:
    axs[0].plot(rz, full_field_model0[:, xpos, 0], color='C2', label='init model')
    axs[0].plot(rz, full_field_meas[:, xpos, 0], color='C0', label='goal model')
    axs[0].plot(rz, full_field_model[:, xpos, 0], color='C1', label='fitted model')
    axs[0].set_ylabel('Bx [T]')
else:
    axs[0].plot(rz, diffx, color='C2', label='error')
    axs[0].plot(rz[:-1], ix, color='C0', label='integrated error')
    axs[0].set_ylabel('Bx [T]')
axs[0].legend()

if not show_diff:
    axs[1].plot(rz, full_field_model0[:, xpos, 1], color='C2',)
    axs[1].plot(rz, full_field_meas[:, xpos, 1], color='C0',)
    axs[1].plot(rz, full_field_model[:, xpos, 1], color='C1',)
    axs[1].set_ylabel('By [T]')
else:
    axs[1].plot(rz, diffy, color='C2', label='error')
    axs[1].plot(rz[:-1], iy, color='C0', label='integrated error')
    axs[1].set_ylabel('By [T]')

if not show_diff:
    axs[2].plot(rz, full_field_model0[:, xpos, 2], color='C2',)
    axs[2].plot(rz, full_field_meas[:, xpos, 2], color='C0',)
    axs[2].plot(rz, full_field_model[:, xpos, 2], color='C1',)
    axs[2].set_ylabel('Bz [T]')
else:
    axs[2].plot(rz, diffz, color='C2', label='error')
    axs[2].plot(rz[:-1], iz, color='C0', label='integrated error')
    axs[2].set_ylabel('Bz [T]')
axs[2].set_xlabel('z [mm]')


In [ ]:
mag_final = np.zeros((4, 93, 3))
mag_goal = np.zeros((4, 93, 3))
for i, cas in enumerate(cas_names):
    for j, block in enumerate(delta.cassettes_ref[cas].blocks):
       mag_final[i, j, :] = block.magnetization
       mag_goal[i, j, :] = delta2.cassettes_ref[cas].blocks[j].magnetization


In [ ]:
mag = 0
cas = 2
plt.title('x magnetization - CSE')
plt.plot(mag_goal[cas,:, mag], 'o-', label='Goal magnetization')
plt.plot(mag_final[cas,:, mag], '.-', label='Fitted magnetization')
plt.xlabel('Block number')
plt.ylabel('Remanent magnetization [T]')
plt.legend()


## Tests with block displacement in jacobian

In [18]:
cas_2_angle = dict()
cas_2_angle['cid'] = -np.pi/4
cas_2_angle['csd'] = -3*np.pi/4
cas_2_angle['cie'] = np.pi/4
cas_2_angle['cse'] = 3*np.pi/4


In [19]:
def do_block_displacement(delta, block, cas):
    block.shift([0, -delta.gap/2, 0])
    center = np.array(block.center_point)
    center[0] = 0
    center[1] = 0
    block.shift(-center)
    block.rotate([0, 0, 0], [0, 0, 1], cas_2_angle[cas])


In [20]:
def do_block_shift(block, shift_list, cas):
    x = shift_list[0]
    y = shift_list[1]
    shiftb = np.array([x, y])

    # Apply local shift to block
    theta = cas_2_angle[cas]
    M = np.zeros((2, 2))
    M[0, 0] = np.cos(theta)
    M[0, 1] = -np.sin(theta)
    M[1, 0] = np.sin(theta)
    M[1, 1] = np.cos(theta)
    shift = np.dot(M, shiftb)
    block.shift([shift[0], shift[1], 0])


In [36]:
def calculate_jac_elements_all(delta, rx, rz, cas='cid'):
    blocks = [
        delta.cassettes_ref[cas].blocks[block_nr] for block_nr in
        np.arange(0, 8, 1)]
    blocks += [
            delta.cassettes_ref[cas].blocks[-4],
            delta.cassettes_ref[cas].blocks[-2]]
    nr_blocks = len(blocks)

    x = rx
    z = rz

    dmag = 0.04
    dshift = 0.04

    delta_shifts = [np.array([dshift, 0, 0]),
                    np.array([0, dshift, 0])]

    delta_mags = [np.array([dmag, 0, 0]),
                  np.array([0, dmag, 0]),
                  np.array([0, 0, dmag]),
                  ]

    jac_elem = np.zeros((len(z), len(x), len(delta_mags)+len(delta_shifts), nr_blocks, 3))

    z_range = 600  # [mm]
    sel = (z >= z_range/2) | (z <= -z_range/2)

    for k, block in enumerate(blocks):
        print('block: ', k)
        for i, x_ in enumerate(x):
            for j, d_mag in enumerate(delta_mags):

                block.magnetization = np.array(block.magnetization) + d_mag/2
                do_block_displacement(delta, block, cas)
                field_p = block.get_field(x=x_, y=0, z=z, nproc=24)


                block.magnetization = np.array(block.magnetization) - d_mag
                do_block_displacement(delta, block, cas)
                field_n = block.get_field(x=x_, y=0, z=z, nproc=24)

                block.magnetization = np.array(block.magnetization) + d_mag/2
                do_block_displacement(delta, block, cas)

                diff = (field_p - field_n)/dmag
                diff[sel] = 0

                jac_elem[:, i, j, k, :] = diff

            for j, d_shift in enumerate(delta_shifts):
                do_block_shift(block, d_shift/2, cas)
                field_p = block.get_field(x=x_, y=0, z=z, nproc=24)

                do_block_shift(block, -d_shift, cas)
                field_n = block.get_field(x=x_, y=0, z=z, nproc=24)

                do_block_shift(block, d_shift/2, cas)

                diff = (field_p - field_n)/dshift
                diff[sel] = 0

                jac_elem[:, i, len(delta_mags) + j, k, :] = diff

    return jac_elem


In [ ]:
nr_points_z = 901
nr_points_x = 3

rx = np.linspace(-1,1,nr_points_x)
rz = np.linspace(-900, 900, nr_points_z)

delta = generate_radia_model()
jac_elem_cid = calculate_jac_elements_all(delta, rx, rz, 'cid')

delta = generate_radia_model()
jac_elem_cie = calculate_jac_elements_all(delta, rx, rz, 'cie')

delta = generate_radia_model()
jac_elem_csd = calculate_jac_elements_all(delta, rx, rz, 'csd')

delta = generate_radia_model()
jac_elem_cse = calculate_jac_elements_all(delta, rx, rz, 'cse')


In [21]:
def calculate_jac_elements_test(delta, rx, rz):
    blocks = [
        delta.cassettes_ref['cid'].blocks[1]]

    nr_blocks = len(blocks)

    # x = rx
    x = [0]
    z = rz

    dshift = 0.02

    delta_shifts = [np.array([dshift, 0, 0]),
                    np.array([0, dshift, 0])]

    jac_elem = np.zeros((len(z), len(x), len(delta_shifts), nr_blocks, 3))
    for k, block in enumerate(blocks):
        do_block_displacement(delta, block)
        print('block: ', k)
        for i, x_ in enumerate(x):


            for j, d_shift in enumerate(delta_shifts):
                print(block.magnetization)
                print(block.center_point)
                do_block_shift(block, d_shift)
                field_p = block.get_field(x=x_, y=0, z=z, nproc=24)

                print(block.center_point)
                do_block_shift(block, -2*d_shift)
                field_n = block.get_field(x=x_, y=0, z=z, nproc=24)

                print(block.center_point)
                do_block_shift(block, d_shift)
                print(block.center_point)

                diff = (field_p - field_n)/(2*dshift)
                half_length = int(len(z)/2)
                diff[0:half_length-600] = 0
                diff[half_length+600:] = 0

                jac_elem[:, i, j, k, :] = diff

    return jac_elem


In [22]:
def transform_cd_2_ce(jac_elem):
    jac_elem[:, :, :, :, :] = jac_elem[:, ::-1, :, :, :]  # Invert x grid

    # delta mx
    jac_elem[:, :, 0, :, 1] = -1*jac_elem[:, :, 0, :, 1]  # Change By signal
    jac_elem[:, :, 0, :, 2] = -1*jac_elem[:, :, 0, :, 2]  # Change Bz signal

    # delta my
    jac_elem[:, :, 1, :, 0] = -1*jac_elem[:, :, 1, :, 0]  # Change Bx signal

    # delta mz
    jac_elem[:, :, 2, :, 0] = -1*jac_elem[:, :, 2, :, 0]  # Change Bx signal

    # delta rx
    jac_elem[:, :, 3, :, 1] = -1*jac_elem[:, :, 3, :, 1]  # Change By signal
    jac_elem[:, :, 3, :, 2] = -1*jac_elem[:, :, 3, :, 2]  # Change Bz signal

    # delta ry
    jac_elem[:, :, 4, :, 0] = -1*jac_elem[:, :, 4, :, 0]  # Change Bx signal

    return jac_elem


In [23]:
def transform_ci_2_cs(jac_elem):
    # delta mx
    jac_elem[:, :, 0, :, 0] = -1*jac_elem[:, :, 0, :, 0]  # Change Bx signal
    jac_elem[:, :, 0, :, 2] = -1*jac_elem[:, :, 0, :, 2]  # Change Bz signal

    # delta my
    jac_elem[:, :, 1, :, 1] = -1*jac_elem[:, :, 1, :, 1]  # Change By signal

    # delta mz
    jac_elem[:, :, 2, :, 1] = -1*jac_elem[:, :, 2, :, 1]  # Change By signal

    # delta rx
    jac_elem[:, :, 3, :, 0] = -1*jac_elem[:, :, 3, :, 0]  # Change Bx signal
    jac_elem[:, :, 3, :, 2] = -1*jac_elem[:, :, 3, :, 2]  # Change Bz signal

    # delta ry
    jac_elem[:, :, 4, :, 1] = -1*jac_elem[:, :, 4, :, 1]  # Change By signal

    return jac_elem


In [24]:
def transform_cid_2_cse(jac_elem):
    jac_cie = transform_cd_2_ce(jac_elem)  # Transform to cie
    jac_cse = transform_ci_2_cs(jac_cie)  # Transform to cse
    return jac_cse


In [25]:
def create_pre_jacobian(jac_elem):
    cid = jac_elem.copy()
    cie = jac_elem.copy()
    csd = jac_elem.copy()
    cse = jac_elem.copy()

    csd = transform_ci_2_cs(csd)
    cie = transform_cd_2_ce(cie)
    cse = transform_cid_2_cse(cse)

    pre_jacobians = {
                     'cse': cse,
                     'csd': csd,
                     'cie': cie,
                     'cid': cid,
                    }
    return pre_jacobians


In [26]:
def get_block_type(block_number, nr_blocks):
    if block_number <= 3:
        block_type = block_number
    elif block_number <= nr_blocks-5:
        block_type = (block_number-4) % 4 + 4
    else:
        cont = -1*(nr_blocks-block_number-4)
        if cont % 2 != 0:
            block_type = nr_blocks-block_number-1
        elif cont == 0:
            block_type = 8
        else:
            block_type = 9
    return block_type


In [27]:
def create_jacobian(pre_jacobians, delta, rx, rz):
    nr_blocks = delta.cassettes_ref['cid'].nr_blocks
    jacobian = np.zeros((len(rz)*len(rx)*3, 4*nr_blocks*5))
    cas_names = get_cassettes_names(delta)
    for i, cas_name in enumerate(cas_names):
        cas_shift = delta.cassettes_ref[cas_name].center_point[-1]
        for j, block in enumerate(delta.cassettes_ref[cas_name].blocks):
            block_shift = block.center_point[-1]
            bl_type = get_block_type(j, nr_blocks)
            idx = np.argmin(np.abs(rz-(block_shift + cas_shift)))
            shift = idx - np.argmin(np.abs(rz))
            shifted_prejac = np.roll(pre_jacobians[cas_name], shift, 0)
            for mag in np.arange(5):
                column = 5*i*nr_blocks + 5*j + mag
                jacobian[:, column] = shifted_prejac[:, :, mag, bl_type, :].ravel(order='F')
    return jacobian


In [28]:
def calc_inverse_jacobian(jacobian):
    u, s, vt = np.linalg.svd(jacobian, full_matrices=False)
    tol_svals = 1e-5
    sel_svals = abs(s) > tol_svals
    ismat = np.zeros(s.shape)
    ismat[sel_svals] = 1/s[sel_svals]
    ismat = np.diag(ismat)
    invmat = np.dot(np.dot(vt.T, ismat), u.T)
    return invmat, u, s, vt


In [29]:
def normalize_jacobian(jac_elem):
    jac_elem = np.swapaxes(jac_elem, -1, 2)
    shape = jac_elem.shape

    aux_elem = jac_elem.reshape(
        shape[0]*shape[1]*shape[2]*shape[3], 5, order='F')

    std_mag = np.std(
        np.concatenate((aux_elem[:, 0],aux_elem[:, 1],aux_elem[:, 2])))
    convx = std_mag/np.std(aux_elem[:, 3])
    convy = std_mag/np.std(aux_elem[:, 4])
    aux_elem[:, 3] *= convx
    aux_elem[:, 4] *= convy

    aux_elem = aux_elem.reshape(shape, order='F')
    jac_elem = np.swapaxes(aux_elem, -1, 2)
    return  jac_elem, convx, convy


In [30]:
imaids.utils.set_len_tol(5e-10, 5e-10)
delta = generate_radia_model(phase=0, dgv=0)
cas_names = get_cassettes_names(delta)

fmap = get_fieldmap(phase=0, dgv=0)
rx = fmap.rx
ry = fmap.ry
rz = fmap.rz

# jac_elem = calculate_jac_elements_all(delta, rx, rz)
# save_pickle(jac_elem, 'jac_elements_block_disp', overwrite=True)


ID4818


In [37]:
# jac_elem = load_pickle('jac_elements_block_disp.pickle')
nr_points_z = 901
nr_points_x = 3

rx = np.linspace(-1,1,nr_points_x)
rz = np.linspace(-900, 900, nr_points_z)

jac_elem = calculate_jac_elements_all(delta, rx, rz)
jac_elem, convx, convy = normalize_jacobian(jac_elem)
pre_jacobians = create_pre_jacobian(jac_elem)

delta = generate_radia_model(phase=0, dgv=0)
jacobian = create_jacobian(pre_jacobians, delta, rx, rz)
invmat, u, s, vt = calc_inverse_jacobian(jacobian)


block:  0
block:  1
block:  2
block:  3
block:  4
block:  5
block:  6
block:  7
block:  8
block:  9


In [ ]:
plt.plot(jac_elem[:, 1, 0, 9, 2])


In [ ]:
plt.plot(np.log10(s), '.')


In [ ]:
a = np.array([1, 2, 3, 4])
sel = (a < 2) | (a > 3)
print(sel)


In [43]:
# Code for tests with other model
phase, dgv = 0, 0
delta2 = generate_radia_model(phase=phase, dgv=dgv, trf_on_blocks=False)
print(cas_names)
mag_dict = dict()
mag_error = dict()
error_dict = dict()
for i, cas_name in enumerate(cas_names):
        mag_list = list()
        mag_error_list = list()
        error_list = list()
        for j, block in enumerate(delta2.cassettes_ref[cas_name].blocks):

                error = 1*np.random.rand(3)
                mag_list.append(1e0*np.array(block.magnetization) + error)
                mag_error_list.append(error)
                error_list.append(list(0*np.random.rand(2)) + [0])

        mag_dict[cas_name] = mag_list
        mag_error[cas_name] = mag_error_list
        error_dict[cas_name] = error_list
delta2.create_radia_object(magnetization_dict=mag_dict,
                           position_err_dict=error_dict)
delta2.set_cassete_positions(dp=phase, dgv=dgv)
full_field_meas = get_full_field_model(rx, rz, delta2)
full_field_meas = np.swapaxes(full_field_meas, 0, 1)


['cse', 'csd', 'cie', 'cid']


In [31]:
delta = generate_radia_model()
full_field_model0 = get_full_field_model(rx, rz, delta)
full_field_model0 = np.swapaxes(full_field_model0, 0, 1)


In [ ]:
%matplotlib inline
delta = generate_radia_model()
jac_elem = calculate_jac_elements_test(delta, rx, rz)
jac_elem.shape
plt.plot(jac_elem[:, 0, 1, 0, 2])


In [ ]:
delta = generate_radia_model()
block = delta.cassettes_ref['cid'].blocks[1]
block.shift([0, -delta.gap/2, 0])
print(block.center_point)

field0 = block.get_field(x=0, y=0, z=rz, nproc=24)

block.shift([0.01, 0, 0])
fieldpx = block.get_field(x=0, y=0, z=rz, nproc=24)
block.shift([-0.02, 0, 0])
fieldnx = block.get_field(x=0, y=0, z=rz, nproc=24)

block.shift([0.01, 0, 0])

block.shift([0, 0.01, 0])
fieldpy = block.get_field(x=0, y=0, z=rz, nproc=24)
block.shift([0, -0.02, 0])
fieldny = block.get_field(x=0, y=0, z=rz, nproc=24)

block.shift([0, 0.01, 0])
print(block.center_point)


In [ ]:
fieldx = block.get_field(x=rx, y=0, z=block.center_point[-1], nproc=24)
plt.plot(rx, fieldx[:, 2], '-o')


In [ ]:
plt.plot(fieldpx[:, 2]-fieldnx[:, 2])
# plt.plot(fieldpy[:, 2]-fieldny[:, 2])


In [ ]:
delta.cassettes_ref['cid'].blocks[1].magnetization


In [ ]:
bx = jac_elem[:, 5, 3, 1, 0]
by = jac_elem[:, 5, 3, 1, 1]
bz = jac_elem[:, 5, 3, 1, 2]
plt.plot(bx, label='bx')
plt.plot(by, label='by')
plt.plot(bz, label='bz')
plt.legend()
plt.show()


In [ ]:
cas_nr = 3
nr_block = 40
par = 4

axis_nr = 0

plt.figure(3)
for nr_block in np.arange(0, 92, 1):
    b = jacobian[((3*nr_points_z)*axis_nr):((3*nr_points_z)*(axis_nr+1)), cas_nr*5*93 + nr_block*5 + par]
    bx = b[0*nr_points_z:1*nr_points_z]
    by = b[1*nr_points_z:2*nr_points_z]
    bz = b[2*nr_points_z:3*nr_points_z]
    plt.plot(rz, bx, color=plt.cm.jet(nr_block/93), alpha=1, label='bx')
    plt.plot(rz, by, color=plt.cm.jet(nr_block/93), alpha=.6, label='by')
    plt.plot(rz, bz, color=plt.cm.jet(nr_block/93), alpha=.2, label='bz')
    # plt.legend()
plt.show()


In [42]:
# Code for tests with other model
phase, dgv = 0, 0
delta = generate_radia_model(phase=phase, dgv=dgv, trf_on_blocks=False)
print(cas_names)
mag_dict = dict()
mag_error = dict()
error_dict = dict()
for i, cas_name in enumerate(cas_names):
        mag_list = list()
        mag_error_list = list()
        error_list = list()
        for j, block in enumerate(delta.cassettes_ref[cas_name].blocks):

            mag_list.append(1e0*np.array(block.magnetization))
            mag_error_list.append(error)
            error_list.append(list(0*np.random.rand(2)) + [0])

        mag_dict[cas_name] = mag_list
        mag_error[cas_name] = mag_error_list
        error_dict[cas_name] = error_list
delta.create_radia_object(magnetization_dict=mag_dict)


['cse', 'csd', 'cie', 'cid']


In [44]:
delta_nominal = generate_radia_model()
full_field_model0 = get_full_field_model(rx, rz, delta_nominal)
full_field_model0 = np.swapaxes(full_field_model0, 0, 1)
delta_field = full_field_meas-full_field_model0


In [45]:
delta_field = full_field_meas-full_field_model0


In [46]:
plt.figure(1)
plt.plot(full_field_meas[:, 1, 2], label='with errors')
plt.plot(full_field_model0[:, 1, 2], label='nominal')
plt.legend()


In [47]:
deltas = np.zeros(93*4*5)
for i, key in enumerate(mag_error.keys()):
    for blck in np.arange(len(mag_error[key])):
        for mag in np.arange(len(mag_error[key][blck])):
            deltas[i*93*5 + blck*5 + mag] = mag_error[key][blck][mag]
        for shift in np.arange(len(error_dict[key][blck])-1):
            if shift == 0:
                fac = convx
            elif shift == 1:
                fac = convy
            deltas[i*93*5 + blck*5 + 3 + shift] = error_dict[key][blck][shift]*1/fac


In [48]:
deltas[0:100]


array([0.94147653, 0.94395535, 0.98577953, 0.        , 0.        ,
       0.28298431, 0.9138263 , 0.69326504, 0.        , 0.        ,
       0.58133231, 0.21176425, 0.43282289, 0.        , 0.        ,
       0.27748067, 0.96095648, 0.44009334, 0.        , 0.        ,
       0.53744307, 0.25598178, 0.91923783, 0.        , 0.        ,
       0.39169153, 0.36429054, 0.27470856, 0.        , 0.        ,
       0.48646853, 0.34214933, 0.27574224, 0.        , 0.        ,
       0.33500311, 0.93236628, 0.44031442, 0.        , 0.        ,
       0.08196264, 0.81305604, 0.53920191, 0.        , 0.        ,
       0.48190979, 0.89961741, 0.37767544, 0.        , 0.        ,
       0.00896286, 0.07630972, 0.21001885, 0.        , 0.        ,
       0.21641603, 0.59394108, 0.17376179, 0.        , 0.        ,
       0.79921787, 0.80744554, 0.90085883, 0.        , 0.        ,
       0.59103326, 0.82569022, 0.48143994, 0.        , 0.        ,
       0.00099007, 0.98269082, 0.89772395, 0.        , 0.     

In [49]:
field0 = np.dot(jacobian, deltas)


In [50]:
field = field0.reshape(full_field_meas.shape, order='F')


In [51]:
field = field.ravel(order='F')
delta_field = delta_field.ravel(order='F')


In [86]:
%matplotlib qt5
diff = field-delta_field
std = np.std(diff)
label = 'Difference -> std = {:.2f} G'.format(1e4*std)
data = load_pickle('data')


rz_n = data['rz']
delta_field_jac = data['delta_field_jac']
delta_field_model = data['delta_field_model']
diff_n = delta_field_jac - delta_field_model
std = np.std(diff_n)
label_n = 'Difference -> std = {:.2f} G'.format(1e4*std)

fig, axs = plt.subplots(2, 1,  sharex=True)
fig.set_figwidth(8)
fig.set_figheight(6)
fig.suptitle(r'Fitting $\Delta$B vs Model $\Delta$B - lin H; K=0')
axs[0].plot(rz, delta_field[-1-len(rz):-1], label='Perturbed model', color='C0')
axs[0].plot(rz, field[-1-len(rz):-1], label='Fitting', color='C1')
axs[0].plot(rz, diff[-1-len(rz):-1], label=label, color='C2')
axs[1].plot(rz, delta_field_model[-1-len(rz):-1], label='Perturbed model', color='C0')
axs[1].plot(rz_n, delta_field_jac[-1-len(rz):-1], label='Fitting', color='C1')
axs[1].plot(rz, diff_n[-1-len(rz):-1], label=label_n, color='C2')
axs[0].set_ylabel(r'$\Delta$Bz [T]')
axs[1].set_ylabel(r'$\Delta$Bz [T]')
axs[1].set_xlabel('rz [mm]')
axs[0].legend(loc='upper left')
axs[1].legend(loc='upper left')
plt.savefig('New_jacobian_test.png')


In [39]:
%matplotlib qt5
plt.figure(2)
plt.plot(field, label='jacobian')
plt.plot(delta_field, label='model')
plt.plot(field-delta_field, label='diff')
plt.legend()


In [ ]:
delta_test = np.dot(invmat, delta_field)
field2 = np.dot(jacobian, delta_test)


In [ ]:
plt.plot(field)
plt.plot(field2)
plt.plot(field2-field)


In [ ]:
plt.plot(delta_test, '-o')


In [ ]:
%matplotlib qt5
plt.plot(deltas, 'o-')
plt.plot(delta_test, '.-')


In [ ]:
#  create radia model
phase, dgv = 0, 0
imaids.utils.set_len_tol(5e-10, 5e-10)
delta = generate_radia_model(phase=phase, dgv=dgv)
cas_names = get_cassettes_names(delta)
nr_blocks = delta.cassettes_ref['cid'].nr_blocks

residue = list()
for k in range(10):
    print('iteraction: ', k)
    field_diff = calc_field_difference(full_field_meas)
    residue_ = np.std(field_diff)
    residue.append(residue_)
    print(residue_)
    # idx = np.argwhere(np.abs(field_diff) < 1e-6)
    # field_diff[idx] = 0

    deltas = 0.7*np.dot(invmat, field_diff)
    deltas = deltas.reshape((len(cas_names), nr_blocks, 5))
    mag_dict = dict()
    error_dict = dict()
    cas_names = get_cassettes_names(delta)
    for i, cas_name in enumerate(cas_names):
        mag_list = list()
        error_list = list()
        for j, block in enumerate(delta.cassettes_ref[cas_name].blocks):
            mag_list.append(np.array(block.magnetization) + deltas[i, j, 0:3])

            previous_error = np.array(delta.position_err_dict[cas_name][j])
            error_new = np.array([deltas[i, j, 3]*convx, deltas[i, j, 4]*convy, 0]) + previous_error

            error_list.append(list(error_new))

        mag_dict[cas_name] = mag_list
        error_dict[cas_name] = error_list
    delta.create_radia_object(magnetization_dict=mag_dict,
                              position_err_dict=error_dict)
    delta.set_cassete_positions(dp=phase, dgv=dgv)


In [ ]:
for k in range(5):
    print('iteraction: ', k)
    field_diff = calc_field_difference(full_field_meas)
    residue_ = np.std(field_diff)
    residue.append(residue_)
    print(residue_)
    # idx = np.argwhere(np.abs(field_diff) < 1e-6)
    # field_diff[idx] = 0

    deltas = 0.8*np.dot(invmat, field_diff)
    deltas = deltas.reshape((len(cas_names), nr_blocks, 5))
    mag_dict = dict()
    error_dict = dict()
    cas_names = get_cassettes_names(delta)
    for i, cas_name in enumerate(cas_names):
        mag_list = list()
        error_list = list()
        for j, block in enumerate(delta.cassettes_ref[cas_name].blocks):
            mag_list.append(np.array(block.magnetization) + deltas[i, j, 0:3])

            previous_error = np.array(delta.position_err_dict[cas_name][j])
            error_new = np.array([deltas[i, j, 3]*convx, deltas[i, j, 4]*convy, 0]) + previous_error

            error_list.append(list(error_new))
        mag_dict[cas_name] = mag_list
        error_dict[cas_name] = error_list
    delta.create_radia_object(magnetization_dict=mag_dict,
                              position_err_dict=error_dict)
    delta.set_cassete_positions(dp=phase, dgv=dgv)


In [ ]:
mag_final = np.zeros((4, 93, 3))
mag_goal = np.zeros((4, 93, 3))
for i, cas in enumerate(cas_names):
    for j, block in enumerate(delta.cassettes_ref[cas].blocks):
       mag_final[i, j, :] = block.magnetization
       mag_goal[i, j, :] = delta2.cassettes_ref[cas].blocks[j].magnetization


In [ ]:
mag = 2
cas = 0
plt.title('x magnetization - CSE')
plt.plot(mag_goal[cas,:, mag], 'o-', label='Goal magnetization')
plt.plot(mag_final[cas,:, mag], '.-', label='Fitted magnetization')
plt.xlabel('Block number')
plt.ylabel('Remanent magnetization [T]')
plt.legend()


In [ ]:
delta.position_err_dict['cse'][10]


In [ ]:
error_final = np.zeros((4, 93, 3))
error_goal = np.zeros((4, 93, 3))
for i, cas in enumerate(cas_names):
    for j, block in enumerate(delta.cassettes_ref[cas].blocks):
       error_final[i, j, :] = np.array(delta.position_err_dict[cas][j])
       error_goal[i, j, :] = np.array(delta2.position_err_dict[cas][j])


In [ ]:
dim = 0
cas = 1
# plt.title('x magnetization - CSE')
plt.plot(error_goal[cas,:, dim], 'o-', label='Goal error')
plt.plot(error_final[cas,:, dim], '.-', label='Fitted error')
plt.xlabel('Block number')
plt.ylabel('Block shift [mm]')
plt.legend()


In [ ]:
full_field_model = get_full_field_model(rx, rz, delta)
full_field_model = np.swapaxes(full_field_model, 0, 1)


In [ ]:
plt.plot(full_field_meas[:, 10, 1])
plt.plot(full_field_model[:, 10, 1])


In [19]:
phase = 0
dgv = 0
fmap = get_fieldmap(phase=phase, dgv=dgv)
rx = fmap.rx
ry = fmap.ry
rz = fmap.rz
full_field_meas = get_full_field(fmap)
full_field_meas = np.swapaxes(full_field_meas, 0, 1)


ID4818


In [16]:
delta = generate_radia_model(phase=0, dgv=0)
data = load_pickle('data')
mag_dict = data['mag']
error_dict = data['shift']

delta.create_radia_object(magnetization_dict=mag_dict,
                                  position_err_dict=error_dict)


In [20]:
full_field_model = get_full_field_model(rx, rz, delta)
full_field_model = np.swapaxes(full_field_model, 0, 1)


In [26]:
%matplotlib qt5
plt.plot(full_field_meas[:, 5, 1])
plt.plot(full_field_model[:, 5, 1])
